In [2]:
ruta_data = r'C:\Documents'

In [3]:
path_xlsx_parte = ruta_data + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\parte_diario_pulverizacion.xlsx'

In [4]:
import sys
sys.path.append('../_amigocloud')
from amigocloud import AmigoCloud
import pandas as pd
from shapely import wkb
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
import os

In [5]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [6]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [10]:
consulta = {'query': 'select * from dataset_367119'}
seleccion = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35248/sql', consulta)
datos = seleccion['data']

In [11]:
df_datos = gpd.GeoDataFrame(datos)

In [12]:
df_datos

,id,amigo_id,fecha_registro,referencia,temp_max,temp_min,viento,humedad,observaciones,tipo_cultivo,tipo_desarrollo
0,1,826a8b15d1b64352af7683dd95895287,2025-03-06 03:28:54+00:00,0101000020E61000005A9B21B0CD7C4FC0F71F990E9D3A...,25.0,20.0,5.0,80.0,Sin observaciones,CAÑA DE AZUCAR,MADURACION


In [13]:
list(set(df_datos['id']))

[1]

In [14]:
idd = 1

In [25]:
query = {'query': f'SELECT apli.*, plan.unidad_01, plan.unidad_03, plan.os\
    FROM dataset_367119 apli\
    JOIN dataset_360912 plan\
    ON ST_Intersects(apli.referencia, plan.geometry)\
    where apli.id = {idd}'}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35248/sql', query)
data_apli = select['data']
#df_apli = pd.DataFrame(data_apli)
#df_apli

,id,amigo_id,fecha_registro,referencia,temp_max,temp_min,viento,humedad,observaciones,tipo_cultivo,tipo_desarrollo,unidad_01,unidad_03,os
0,1,826a8b15d1b64352af7683dd95895287,2025-03-06 03:28:54+00:00,0101000020E61000005A9B21B0CD7C4FC0F71F990E9D3A...,25.0,20.0,5.0,80.0,Sin observaciones,CAÑA DE AZUCAR,MADURACION,30,41594,1


In [49]:
dic_apli = data_apli[0]
dic_apli

{'id': 1,
 'amigo_id': '826a8b15d1b64352af7683dd95895287',
 'fecha_registro': '2025-03-06 03:28:54+00:00',
 'referencia': '0101000020E61000005A9B21B0CD7C4FC0F71F990E9D3A31C0',
 'temp_max': 25.0,
 'temp_min': 20.0,
 'viento': 5.0,
 'humedad': 80.0,
 'observaciones': 'Sin observaciones',
 'tipo_cultivo': 'CAÑA DE AZUCAR',
 'tipo_desarrollo': 'MADURACION',
 'unidad_01': 30,
 'unidad_03': 41594,
 'os': 1}

In [65]:
cod_prop = dic_apli['unidad_01']
cod_ca = dic_apli['unidad_03']
oss = dic_apli['os']

In [69]:
print('propiedad:', cod_prop)
print('cañero:', cod_ca)
print('os:', oss)

propiedad: 30
cañero: 41594
os: 1


In [39]:
nulos = df_apli.isnull().sum()
nulos

id                 0
amigo_id           0
fecha_registro     0
referencia         0
temp_max           0
temp_min           0
viento             0
humedad            0
observaciones      0
tipo_cultivo       0
tipo_desarrollo    0
unidad_01          0
unidad_03          0
os                 0
dtype: int64

In [30]:
df_parte = pd.read_excel(path_xlsx_parte)
df_parte.head(3)

,Nº DE LABOR,FECHA,SEMANA,INSTITUCION,COD CANERO,NOMBRE CANERO,COD PROPIEDAD,NOMBRE PROPIEDAD,LOTE,HECTAREA,...,CICLO,ID,OS,TEMP MAX,TEMP MIN,TIPO CULTIVO,TIPO DESARROLLO,NUM VUELOS,HORAS VUELO,TOTAL CAUDAL
0,PA-2025-1,NaN,NaN,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,EP-L31,30.260942,...,2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PA-2025-1,NaN,NaN,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,EP-L16,23.216246,...,4,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PA-2025-1,NaN,NaN,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,ER-L6,19.683505,...,5,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df_parte['FECHA'] = pd.to_datetime(df_parte['FECHA'], errors='coerce')
df_parte['HORA DE INICIO'] = pd.to_datetime(df_parte['HORA DE INICIO'], errors='coerce')
df_parte['HORA FINAL'] = pd.to_datetime(df_parte['HORA FINAL'], errors='coerce')

In [61]:
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'idd'] = dic_apli['id']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'TEMP MAX'] = dic_apli['temp_max']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'TEMP MIN'] = dic_apli['temp_min']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'VIENTO'] = dic_apli['viento']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'HUMEDAD'] = dic_apli['humedad']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'TIPO CULTIVO'] = dic_apli['tipo_cultivo']
df_parte.loc[(df_parte['COD PROPIEDAD'] == dic_apli['unidad_01']) & 
             (df_parte['COD CANERO'] == dic_apli['unidad_03']) & 
             (df_parte['OS'] == dic_apli['os']), 'TIPO DESARROLLO'] = dic_apli['tipo_desarrollo']

In [63]:
df_parte.head(3)

,Nº DE LABOR,FECHA,SEMANA,INSTITUCION,COD CANERO,NOMBRE CANERO,COD PROPIEDAD,NOMBRE PROPIEDAD,LOTE,HECTAREA,...,ID,OS,TEMP MAX,TEMP MIN,TIPO CULTIVO,TIPO DESARROLLO,NUM VUELOS,HORAS VUELO,TOTAL CAUDAL,idd
0,PA-2025-1,NaT,NaN,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,EP-L31,30.260942,...,1,1,25.0,20.0,CAÑA DE AZUCAR,MADURACION,NaN,NaN,NaN,1.0
1,PA-2025-1,NaT,NaN,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,EP-L16,23.216246,...,2,1,25.0,20.0,CAÑA DE AZUCAR,MADURACION,NaN,NaN,NaN,1.0
2,PA-2025-1,NaT,NaN,50,41594,AGROPECUARIA CAMPO DULCE S.R.L.,30,CAMPO DULCE,ER-L6,19.683505,...,3,1,25.0,20.0,CAÑA DE AZUCAR,MADURACION,NaN,NaN,NaN,1.0


In [64]:
output_path = path_xlsx_parte
df_parte.to_excel(output_path, index=False)

# HACER COPIA Y ELIMINAR PARTE DIARIO Y PLANIFICACION DE AMIGOCLOUD

In [70]:
idd

1

In [72]:
#copia y elimina los idd indicados del parte diario de amigocloud
id_proyecto = f'https://app.amigocloud.com/api/v1/projects/35248/sql'
query = f'INSERT INTO dataset_367584 (id, fecha_registro, temp_max, temp_min, viento, humedad, observaciones, tipo_cultivo, tipo_desarrollo, referencia)\
        SELECT id, fecha_registro, temp_max, temp_min, viento, humedad, observaciones, tipo_cultivo, tipo_desarrollo, referencia\
        FROM dataset_367119\
        WHERE id = {idd}'
query_sql = {'query': query}
post = amigocloud.post(id_proyecto, query_sql)
query = {'query': f'DELETE FROM dataset_367119 WHERE id = {idd}'}
ejecutar = amigocloud.post(id_proyecto, query)
print('se proceso IDD:', idd)

AmigoCloudError: 400 Client Error: Bad Request for url: https://app.amigocloud.com/api/v1/projects/35248/sql?token=A:cf5xrxU3HgHj9FkLMiY5wW9LBFmvYDQG2TQ5FY
{"query":"DELETE FROM dataset_367119 WHERE idd = 1","error":"column \"idd\" does not exist"}

In [74]:
#copia y elimina los id indicados de la planificacion semanal de amigocloud
id_proyecto = f'https://app.amigocloud.com/api/v1/projects/35248/sql'
query = f'INSERT INTO dataset_360915 (fecha_registro, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, geometry)\
            SELECT fecha_registro, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, geometry\
            FROM dataset_360912\
            WHERE unidad_01 = {cod_prop} and os = {oss}'
query_sql = {'query': query}
post = amigocloud.post(id_proyecto, query_sql)
query = {'query': f'DELETE FROM dataset_360912 WHERE unidad_01 = {cod_prop} and os = {oss}'}
ejecutar = amigocloud.post(id_proyecto, query)
print('se proceso ID:', idd)

## Copiar ids de PARTE DIARIO a PLANIFICACION

In [87]:
path_planificacion = ruta_data + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\SHP\PLANIFICACION.shp'

In [88]:
gdf_plan = gpd.read_file(path_planificacion)

In [89]:
gdf_plan.loc[(gdf_plan['unidad_01'] == dic_apli['unidad_01']) & 
             (gdf_plan['unidad_03'] == dic_apli['unidad_03']) & 
             (gdf_plan['os'] == dic_apli['os']), 'idd'] = dic_apli['id']

In [90]:
gdf_plan

,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,id,inst,idd,geometry
0,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L31,30.260942,1,2,1,50,1,"POLYGON ((504085.659 8094406.031, 503737.808 8..."
1,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L16,23.216246,1,4,2,50,1,"POLYGON ((504352.053 8094797.970, 504272.088 8..."
2,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,ER-L6,19.683505,1,5,3,50,1,"POLYGON ((503260.583 8094596.306, 503714.735 8..."
3,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L35,7.795719,1,4,4,50,1,"POLYGON ((504693.348 8094607.629, 504673.327 8..."
4,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L34,7.048921,1,5,5,50,1,"POLYGON ((504137.327 8094415.183, 504648.080 8..."
5,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,ER-L5,10.318332,1,5,6,50,1,"POLYGON ((502800.712 8094558.618, 502817.481 8..."
6,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,ER-L9,26.783587,1,3,7,50,1,"POLYGON ((502151.946 8095089.317, 502180.927 8..."
7,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,ER-L15,17.950658,1,2,8,50,1,"POLYGON ((501079.860 8094856.577, 501419.672 8..."
8,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L7,15.371285,1,2,9,50,1,"POLYGON ((501510.045 8095556.280, 501495.788 8..."
9,30,CAMPO DULCE,41594,AGROPECUARIA CAMPO DULCE S.R.L.,EP-L10,26.586339,1,2,10,50,1,"POLYGON ((502549.761 8095663.451, 503187.898 8..."


In [91]:
gdf_plan.dtypes

unidad_01       int64
unidad_02      object
unidad_03       int64
unidad_04      object
unidad_05      object
area          float64
os              int64
soca            int64
id              int64
inst            int64
idd             int64
geometry     geometry
dtype: object

In [92]:
gdf_plan.to_file(path_planificacion, driver='ESRI Shapefile')